In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

In [2]:
# Import cities file as DataFrame
cost_living = pd.read_csv("Resources/Cost of Living.csv")
cost_living.head()

,City,Country,Column12,Column3,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,Tirana,Albania,ALL,Albanian Lek,39.29,11.49,25.93,31.29,29.29,27.96
1,Durres,Albania,ALL,Albanian Lek,31.86,6.18,19.52,24.70,24.10,27.09
2,Algiers,Algeria,DZD,Algerian Dinar,28.46,5.83,17.59,28.68,17.98,22.53
3,Oran,Algeria,DZD,Algerian Dinar,25.66,4.76,15.62,26.77,13.22,20.68
4,Buenos Aires,Argentina,ARS,Argentine Peso,33.00,9.55,21.74,26.74,29.09,28.95


In [3]:
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
cost_living["Lat"] = ""
cost_living["Lng"] = ""
cost_living.head()

,City,Country,Column12,Column3,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Lat,Lng
0,Tirana,Albania,ALL,Albanian Lek,39.29,11.49,25.93,31.29,29.29,27.96,,
1,Durres,Albania,ALL,Albanian Lek,31.86,6.18,19.52,24.70,24.10,27.09,,
2,Algiers,Algeria,DZD,Algerian Dinar,28.46,5.83,17.59,28.68,17.98,22.53,,
3,Oran,Algeria,DZD,Algerian Dinar,25.66,4.76,15.62,26.77,13.22,20.68,,
4,Buenos Aires,Argentina,ARS,Argentine Peso,33.00,9.55,21.74,26.74,29.09,28.95,,


In [5]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in cost_living.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    country = row['Country']
    city = row['City']

    # update address key value
    params['address'] = f"{country},{city}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cost_living.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cost_living.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]



In [11]:
# Visualize to confirm lat lng appear
cost_living.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   City                            518 non-null    object 
 1   Country                         518 non-null    object 
 2   Column12                        518 non-null    object 
 3   Column3                         518 non-null    object 
 4   Cost of Living Index            518 non-null    float64
 5   Rent Index                      518 non-null    float64
 6   Cost of Living Plus Rent Index  518 non-null    float64
 7   Groceries Index                 518 non-null    float64
 8   Restaurant Price Index          518 non-null    float64
 9   Local Purchasing Power Index    518 non-null    float64
 10  Lat                             518 non-null    object 
 11  Lng                             518 non-null    object 
dtypes: float64(6), object(6)
memory usag

In [12]:
cost_living.head()


,City,Country,Column12,Column3,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Lat,Lng
0,Tirana,Albania,ALL,Albanian Lek,39.29,11.49,25.93,31.29,29.29,27.96,41.327546,19.818698
1,Durres,Albania,ALL,Albanian Lek,31.86,6.18,19.52,24.70,24.10,27.09,41.32459,19.456469
2,Algiers,Algeria,DZD,Algerian Dinar,28.46,5.83,17.59,28.68,17.98,22.53,36.753768,3.058756
3,Oran,Algeria,DZD,Algerian Dinar,25.66,4.76,15.62,26.77,13.22,20.68,35.698739,-0.634932
4,Buenos Aires,Argentina,ARS,Argentine Peso,33.00,9.55,21.74,26.74,29.09,28.95,-34.603684,-58.381559


In [16]:
cost_living.rename(columns={"Column12":"Currency Code","Column3":"Currency Name"},inplace=True)
cost_living.head()

,City,Country,Currency Code,Currency Name,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Lat,Lng
0,Tirana,Albania,ALL,Albanian Lek,39.29,11.49,25.93,31.29,29.29,27.96,41.327546,19.818698
1,Durres,Albania,ALL,Albanian Lek,31.86,6.18,19.52,24.70,24.10,27.09,41.32459,19.456469
2,Algiers,Algeria,DZD,Algerian Dinar,28.46,5.83,17.59,28.68,17.98,22.53,36.753768,3.058756
3,Oran,Algeria,DZD,Algerian Dinar,25.66,4.76,15.62,26.77,13.22,20.68,35.698739,-0.634932
4,Buenos Aires,Argentina,ARS,Argentine Peso,33.00,9.55,21.74,26.74,29.09,28.95,-34.603684,-58.381559


In [17]:
cost_living.to_csv("Cost of Living.csv")